# Harmonische Störung

## Vorbereitung

Führen sie zuerst die folgende Zelle aus, um die Simulation vorzubereiten. Sie können diese danach mit dem Pfeil links einklappen. 

In [ ]:
import numpy as np
import time
import plotly.graph_objs as go
from plotly.subplots import make_subplots

def p_if_func(t, w, w_if):
    a = np.sin((w_if+w)*t/2)**2 / (w_if+w)**2
    b = np.sin((w_if-w)*t/2)**2 / (w_if-w)**2
    c = np.sin((w_if-w)*t/2) * np.sin((w_if+w)*t/2) * np.cos(2*w*t) / ((w_if-w) * (w_if+w))
    return 4 * (a + b - 2*c)

def plot_pif(t_max=15, fixed_w=1):
    wsize = 1000
    tsize = 100
    delta_w_array = np.linspace(-fixed_w*3,fixed_w*3,wsize)
    delta_w = fixed_w
    w_array = np.linspace(0,fixed_w*3,wsize)
    w = fixed_w
    t_array = np.linspace(0,t_max,tsize)
    
    p_if_array_vary_wif = np.zeros([tsize,wsize])
    for i in range(tsize):
        p_if_array_vary_wif[i] = p_if_func(t_array[i], w, delta_w_array)
        
    p_if_array_vary_w = np.zeros([tsize,wsize])
    for i in range(tsize):
        p_if_array_vary_w[i] = p_if_func(t_array[i], w_array, delta_w)
        
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=['Für fixes omega = '+str(fixed_w),'Für fixes omega_if = '+str(fixed_w)])

    for i in range(0,tsize):
        fig.add_trace(go.Scatter(x=delta_w_array, y=p_if_array_vary_wif[i], visible=False, name=r'$P_{if}(\omega_{if},t)$'), 1, 1)
        fig.add_trace(go.Scatter(x=w_array, y=p_if_array_vary_w[i], visible=False, name=r'$P_{if}(\omega,t)$'), 1, 2)

    fig.data[0].visible = True
    fig.data[1].visible = True

    steps = []
    for i in range(tsize):
        step = dict(method="update", args=[{"visible": [False] * 2*tsize}], label=str(t_array[i]))
        step["args"][0]["visible"][2*i:2*i+1] = [True,True]
        steps.append(step)


    fig['layout']['xaxis']['title']=r'$\omega_{if}$'
    fig['layout']['xaxis2']['title']=r'$\omega$'
    fig.update_yaxes(range=[0, 10*t_max])
    slider = [dict(active=0, steps=steps, currentvalue={"prefix": "t="})]
    fig.update_layout(sliders=slider, width=1000, height=800)

    fig.show()

## Simulation

Durch das ausführen der nächsten Zellen starten sie die Simulation. Sie können sowohl die Zeitspanne, als auch den Wert der festgehaltenen Frequenz manuell eingeben. 
Anmerkung: Die Fehlermeldung erscheint, weil bei $\omega=\omega_{if}$ durch null dividiert wird.

In [ ]:
plot_pif(t_max=15,fixed_w=1)

# Entwurfsteil - Simulation von Besetzungszahlen in einem Zwei-Niveau System

In [ ]:
def p_if_func(t, w, w_if):
    a = np.sin((w_if+w)*t/2)**2 / (w_if+w+1e-15)**2
    b = np.sin((w_if-w)*t/2)**2 / (w_if-w+1e-15)**2
    c = np.sin((w_if-w)*t/2) * np.sin((w_if+w)*t/2) * np.cos(2*w*t) / ((w_if-w) * (w_if+w)+1e-15)
    return 4 * (a + b - 2*c)

In [ ]:
def p_ii_func(t, w): # Dies ist falsch
    a = np.sin(w*t/2)**4 / (w + 1e-15)**2
    b = np.sin(w*t/2)**2 / (w + 1e-15)
    return 1 + 4*a - 2*b 

In [ ]:
class Übergänge():
    def __init__(self, sample_number=1000, timesteps=200, dt=0.1, w=2, delta_w=3):
        self.sample_number=sample_number
        self.timesteps = timesteps
        self.dt = dt
        self.w = w
        self.delta_w = delta_w
        
        self.external_field = self.get_external_field()
        self.x_values = [i for i in range(self.sample_number)]
        
        self.run_states()
        self.create_frames()
        #self.plot()
        
    def get_external_field(self):
        return [np.sin(self.w * self.dt * t) for t in range(self.timesteps)]    
    
    def p_if_func(self, t):
        a = np.sin((self.delta_w + self.w) * t/2)**2 / (self.delta_w + self.w + 1e-10)**2
        b = np.sin((self.delta_w - self.w) * t/2)**2 / (self.delta_w - self.w + 1e-10)**2
        c = (np.sin((self.delta_w - self.w) * t/2) * np.sin((self.delta_w + self.w) * t/2) *
             np.cos(2 * self.w * t) / ((self.delta_w - self.w) * (self.delta_w + self.w) + 1e-10) )
        return 4 * (a + b - 2*c)
    
    def p_ii_func(self, t):
        a = np.sin(self.w * t/2)**4 / (self.w + 1e-10)**2
        b = np.sin(self.w * t/2)**2 / (self.w + 1e-10)
        return 1 + 4*a - 2*b
    
    def timestep(self, initial, t): # Initial is 1d array
        p_if = self.p_if_func(t)
        p_ii = self.p_ii_func(t)

        p_if_norm = p_if / (p_if+p_ii)
        p_ii_norm = p_ii / (p_if+p_ii)

        change_state = np.random.binomial(1, p_if_norm)
        final = initial

        for i in range(initial.size):
            if change_state[i]==1:
                final[i] = 1 - initial[i] # Bit flip
                t[i] = 0 # Reset time after state change
        return final # reset time since change
    
    def run_states(self):
        states = np.zeros(self.sample_number)
        self.position_history = np.array([states])
        self.total_history = []
        
        t = self.dt * np.ones(self.sample_number)
        for i in range(self.timesteps):
            states = self.timestep(states, t)
            self.position_history = np.append(self.position_history, np.array([states]), axis=0)
            self.total_history.append([self.sample_number - states.sum(), states.sum()])
            t += self.dt
    
    def create_frames(self):
        self.frames = []
        for i in range(self.timesteps):
            self.frames.append(dict(name=i, data=[go.Scatter(x=self.x_values, y=self.position_history[i], mode='markers'),
                                                  go.Scatter(x=[0,0], y=[self.external_field[i],0]), 
                                                  go.Bar(x=['0','1'], y=self.total_history[i])],
                                    traces = [0,1,2]
            ))
            
            
    def plot(self):
        
        fig = make_subplots(rows=2, cols=2, subplot_titles=['Title 1', 'External Field', 'Title 3'], 
                            specs=[[{"colspan": 2}, None],[{}, {}]])

        fig.add_trace(go.Scatter(x=self.x_values, y=self.position_history[0], mode='markers'), row=1, col=1)

        fig.add_trace(go.Scatter(x=[0,0], y=[0,0], marker_size=[10,5], line_dash='dot'), row=2, col=1)

        fig.add_trace(go.Bar(x=['0','1'], y=[self.sample_number, 0]), row=2, col=2)

        fig.update_layout(yaxis2=dict(range=[-1.05, 1.05], autorange=False), xaxis2_showticklabels=False)
        fig.update_layout(yaxis3=dict(range=[0, self.sample_number], autorange=False))

        updatemenus = [dict(type='buttons',
                            buttons=[dict(label='Play',
                                          method='animate',
                                          args=[[f'{k}' for k in range(self.timesteps)], 
                                                 dict(frame=dict(duration=400, redraw=False), 
                                                      transition=dict(duration=300),
                                                      #easing='cubic-in-out',
                                                      #fromcurrent=True,
                                                      #mode='immediate'
                                                                         )])],
                            direction= 'left', 
                            pad=dict(r= 10, t=85), 
                            showactive =True, x= 0.1, y= 0, xanchor= 'right', yanchor= 'top')
                           ]

        fig.update(frames=self.frames),
        fig.update_layout(updatemenus=updatemenus);

        fig.update_layout(width=950, height=800)

        fig.add_trace( go.Scatter(x=[0,self.sample_number], y=[1,1], mode='lines'), row=1, col=1 )
        fig.add_trace( go.Scatter(x=[0,self.sample_number], y=[0,0], mode='lines'), row=1, col=1 )
        fig.show()  

In [ ]:
obj=Übergänge(sample_number=1000, timesteps=100, dt=0.1, w=1, delta_w=3)

In [ ]:
obj.plot()